In [16]:
import sys
from collections import defaultdict, OrderedDict
import pprint as pp
import numpy as np
import pandas as pd
import scipy.optimize as optimize

In [17]:
df = pd.read_csv('task_cpu_90d_1.csv', names=['jeditaskid', 'processingtype', 'transformation', 'atlrel', 'site',
                                                               'cputype', 'njobs', 'cpuevt_avg', 'cpuevt_rms', 'wallevt_avg',
                                                               'wallevt_rms', 'cpu', 'wc', 'cores', 'jevts', 'eff'])

In [18]:
print df[0:10]

   jeditaskid processingtype transformation        atlrel            site  \
0     7965039           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
1     7981879           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
2     7915139           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
3     7915930           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
4     7982179           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
5     8536082           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
6     8428223           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
7     8278972           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
8     7982154           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   
9     8274093           pile     Reco_tf.py  Atlas-20.7.5  BNL_PROD_MCORE   

                                             cputype  njobs  cpuevt_avg  \
0  s+Intel(R) Xeon(R) CPU E5-2660 v3 @ 2.60GHz 25...   2590   46.224859   
1 

In [19]:
grouped = df[df.processingtype == 'evgen'].groupby(['site'], as_index=False)
tot_wc = grouped['wc'].sum()
a = tot_wc.sort_values('wc', ascending=False)
bigtotal_wc = a.wc.sum()
big_sites = a[a.wc > bigtotal_wc * 0.001].site.tolist()
print big_sites

['MWT2_SL6', 'FZK-LCG2', 'CERN-PROD-preprod', 'DESY-HH', 'INFN-T1', 'UKI-SCOTGRID-GLASGOW_SL6', 'UKI-LT2-Brunel_SL6', 'SLACXRD', 'praguelcg2', 'CA-VICTORIA-WESTGRID-T2', 'CERN-PROD', 'UKI-LT2-QMUL_SL6', 'TOKYO', 'OU_OCHEP_SWT2', 'UKI-NORTHGRID-LANCS-HEP_SL6', 'BNL_PROD', 'SWT2_CPB', 'Lucille_CE', 'IFIC', 'LPC', 'pic', 'RAL-LCG2_SL6', 'AGLT2_SL6', 'GRIF-IRFU', 'UNIBE-LHEP', 'DESY-HH_Condor', 'IHEP_PROD', 'UKI-LT2-RHUL_SL6', 'UNI-FREIBURG', 'JINR_PROD', 'RRC-KI-T1', 'CPPM', 'INFN-LECCE', 'CYFRONET-LCG2', 'INFN-NAPOLI-ATLAS', 'BU_ATLAS_Tier2_SL6', 'INFN-FRASCATI', 'GRIF-LPNHE', 'LRZ-LMU', 'Taiwan-LCG2_VL', 'NIKHEF-ELPROD', 'LRZ-LMU_C2PAP', 'wuppertalprod', 'Australia-ATLAS', 'UKI-SOUTHGRID-RALPP_SL6', 'Taiwan-LCG2', 'FMPhI-UNIBA', 'INFN-T1-BA', 'GRIF-LAL_HTCondor', 'IFAE', 'UNI-DORTMUND', 'CA-MCGILL-CLUMEQ-T2', 'UKI-SOUTHGRID-OX-HEP_SL6', 'DESY-ZN', 'TRIUMF', 'UKI-SCOTGRID-ECDF_SL6', 'UKI-NORTHGRID-LIV-HEP_SL6', 'INFN-ROMA1', 'ROMANIA07', 'IN2P3-LPSC', 'GoeGrid', 'UTA_SWT2', 'UKI-NORTHGRI

In [28]:
grouped = df.groupby(['processingtype', 'jeditaskid', 'site'], as_index=False, sort=False)

data = defaultdict(OrderedDict)
for (p, j, s), g in grouped:
    if s not in big_sites: continue
    if p != 'evgen': continue
    x = np.average(g.cpuevt_avg, weights=g.wc)
    data[j][s] = x

2362


In [30]:
# Remove from data tasks with only one site
for (k, v) in data.items():
    if len(v) == 1:
        del data[k]

In [31]:
# Replace site names with numbers
i = -1
sites = set()
smap = dict()
for (k, v) in data.items():
    for s in v.keys():
        if s not in sites:
            i += 1
            smap[s] = i
            sites.add(s)
        data[k][smap[s]]= data[k][s]
        del data[k][s]

nsites = len(smap)
print len(sites)
print 'There are %s sites in the fit' % nsites


119
There are 119 sites in the fit


In [32]:
def func2(k):
    total = 0.
    for _, task in data.items():
        kf = np.array([k[i] for i in task.keys()])
        v = np.array(task.values())
        kf2 = np.roll(kf, -1)
        v2 = np.roll(v, -1)
        ratio = (v2 / v - kf / kf2)**2
        ratio[np.isnan(ratio)] = 0.
        ratio[np.isinf(ratio)] = 0.
        delta = np.sum(ratio[:-1])
        total += delta
    return total

In [33]:
def grad(k):
    grad = np.zeros(len(k))
    for _, task in data.items():
        ns = np.array(task.keys())
        kf = np.array([k[i] for i in task.keys()])
        v = np.array(task.values())
        for i in range(len(task)):
            if i == len(task) - 1:
                x1 = 0.
            else:
                x1 = -2 * (v[i+1] / v[i] - kf[i] / kf[i+1]) / kf[i+1]
            if np.isnan(x1) or np.isinf(x1): x1 = 0.
            if i == 0:
                x2 = 0.
            else:
                x2 = 2 * (v[i] / v[i-1] - kf[i-1] / kf[i]) * kf[i-1] / kf[i]**2
            if np.isnan(x2) or np.isinf(x2): x2 = 0.
            grad[ns[i]] += (x1 + x2)
    return grad

In [40]:
k_ini = np.ones(nsites)   # Initialises CPU factors for all sites to one

print 'Initial function value: ', func2(k_ini)

Initial function value:  1358.22164136


In [41]:
check = optimize.check_grad(func2, grad, k_ini)
print check

0.000445465315136


In [42]:
ref = smap['CERN-PROD']
cons = ({'type': 'eq', 'fun': lambda x: x[ref] - 1})

In [43]:
result = optimize.minimize(func2, k_ini, method='SLSQP', jac=grad, constraints=cons, options={'disp': True, 'maxiter': 2000})

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 535.711820464
            Iterations: 124
            Function evaluations: 407
            Gradient evaluations: 124


In [46]:
k = result.x

In [48]:
for s in sorted(smap.keys()):
    print '%s\t%f' % (s, k[smap[s]])

AGLT2_SL6	1.097346
ATLAS_OPP_OSG	1.107943
Australia-ATLAS	1.281199
BEIJING	1.859081
BNL_ATLAS_2	1.281055
BNL_LOCAL	1.428314
BNL_PROD	1.029025
BU_ATLAS_Tier2_SL6	1.186484
CA-MCGILL-CLUMEQ-T2	1.502764
CA-SCINET-T2	1.312953
CA-VICTORIA-WESTGRID-T2	1.486345
CERN-P1_MCORE	0.864210
CERN-PROD	1.000000
CERN-PROD-preprod	1.104665
CERN-PROD_MCORE	1.512667
CONNECT	1.303786
CPPM	1.111987
CSCS-LCG2	1.650777
CYFRONET-LCG2	0.951783
DCSC	0.790374
DESY-HH	0.834236
DESY-HH_Condor	1.278505
DESY-HH_MCORE	0.865022
DESY-HH_MCORE_Condor	1.136211
DESY-ZN	1.669718
EELA-UTFSM	1.020179
FMPhI-UNIBA	1.148152
FZK-LCG2	1.235239
FZK-LCG2_MCORE_LO	0.928685
GR-12-TEIKAV	0.985807
GRIF-IRFU	1.105371
GRIF-LAL_HTCondor	1.108777
GRIF-LAL_HTCondor_MCORE	1.148587
GRIF-LPNHE	1.038232
GoeGrid	1.614050
HPC2N	0.845837
IAAS	1.352306
IFAE	1.263098
IFIC	1.177629
IHEP_PROD	0.951215
IL-TAU-HEP	1.295819
IL-TAU-HEP_MCORE	1.252557
IN2P3-CC	1.127687
IN2P3-CC-T3_VM01	1.125218
IN2P3-CC_MCORE_HIMEM	1.163077
IN2P3-CC_VVL	1.053145
IN2P3-LPSC	1